In [1]:
# Python 3.x
import re
import math

def Input(day):
    "Open this day's input file."
    filename = 'inputs/{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        print("Oops, couldn't open input")


# Day 1 - Inverse Captcha
## Part 1

> The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

This could be done by reading the digits into an array, then reducing them. As a Python noob, I read that for clarity, list comprehensions are preferred over lambdas, e.g. see Guido van Rossum's [post](http://www.artima.com/weblogs/viewpost.jsp?thread=98196) from 2005. However, list comprehensions produce lists, whereas reduce produces a single output. Van Rossum concedes that `reduce()` may be used when the operator is associative, so I'm going to go ahead with `reduce()`.

In [2]:
from functools import reduce

In [3]:
def chars_to_int_array(str):
    "Given a string of digits, returns them as an array of integers"
    return [int(ch) for ch in str]

def circular_list_next_item(list, index, offset=1): 
    "Returns the list item that is 'offset' elements away after the one given by the index; the item after the last item is the first item"
    return list[(index + offset) % len(list)]

def inverse_captcha_1(arr):
    "Returns sum of digits in array that match the next digit, treating list as circular."
    return reduce(lambda sum, i: sum + (arr[i] if circular_list_next_item(arr, i) == arr[i] else 0), range(len(arr)), 0)
    

assert inverse_captcha_1(chars_to_int_array("1122")) == 3
assert inverse_captcha_1(chars_to_int_array("1111")) == 4
assert inverse_captcha_1(chars_to_int_array("1234")) == 0
assert inverse_captcha_1(chars_to_int_array("91212129")) == 9

inverse_captcha_1(chars_to_int_array(Input(1).read().rstrip("\n")))

1141

## Part 2

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. Fortunately, your list has an even number of elements

In [4]:
def inverse_captcha_2(arr):
    "Returns sum of digits in array that match the digit that is halfway around, treating list as circular."
    return reduce(lambda sum, i: sum + (arr[i] if circular_list_next_item(arr, i, len(arr)//2) == arr[i] else 0), range(len(arr)), 0)

assert inverse_captcha_2(chars_to_int_array("1212")) == 6
assert inverse_captcha_2(chars_to_int_array("1221")) == 0
assert inverse_captcha_2(chars_to_int_array("123425")) == 4
assert inverse_captcha_2(chars_to_int_array("123123")) == 12
assert inverse_captcha_2(chars_to_int_array("12131415")) == 4

inverse_captcha_2(chars_to_int_array(Input(1).read().rstrip("\n")))

950

# Day 2

## Part 1
> The spreadsheet consists of rows of apparently-random numbers. To make sure the recovery process is on the right track, they need you to calculate the spreadsheet's checksum. For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

Initially, I thought I could iterate over each row, keep track of the max and min and subtract. But it's a lot easier to just call the built-in `max` and `min`. Both methods have `O(n)` running time.

In [5]:
def line_to_int_array(line):
    "Given a string with whitespace-separated numbers, parse it as an array of integers"
    return [int(n) for n in line.split()]

def lines_to_int_arrays(lines):
    "Given a multi-line string, each with whitespace-separated numbers, return an array of arrays of integers"
    return [line_to_int_array(L) for L in lines]
    
def max_subtract_min(L):
    "Return the largest element minus the smallest element"
    return max(L) - min(L)

assert(sum([max_subtract_min(L) for L in lines_to_int_arrays("5 1 9 5\n7 5 3\n2 4 6 8".split("\n"))])) == 18

sum([max_subtract_min(L) for L in lines_to_int_arrays(Input(2).readlines())])

37923

## Part 2
> It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

I had a lot of ideas of how to solve this. Clearly, we need to consider all pairs of numbers within each row. Sorting each row's numbers first, would simplify things a little, because we could always divide the smaller into the larger, by having two loops, one from each end. Or, perhaps we could apply a modulo operator, look for the single entry with zero, but that's wasteful, as we don't abort early. 

In the end, I preferred the simpler double for-loop, and having to compute the max/min within the inner block, since it didn't requiring any sorting, or reversing.

In [6]:
def quotient_of_divisible_elements(L):
    "Given a list of numbers in which there are two numbers where one exactly divides the other, return the quotient between them."
    for i in range(0, len(L)-1):
        for j in range(i+1, len(L)):
            p = max(L[i], L[j])
            q = min(L[i], L[j])
            if p % q == 0:
                return p // q

assert(sum([quotient_of_divisible_elements(L) for L in lines_to_int_arrays("5 9 2 8\n9 4 7 3\n3 8 6 5".split("\n"))])) == 9

sum([quotient_of_divisible_elements(L) for L in lines_to_int_arrays(Input(2).readlines())])

263

# Day 3
## Part 1
> Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```
> requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the [Manhattan Distance](https://en.wikipedia.org/wiki/Taxicab_geometry) between the location of the data and square 1.
> How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?


At first, I thought about all the different ways I could represent this spiral structure, and be able to get its grid coordinates in an efficient way. After racking my brain for a bit, I noticed a pattern in the bottom right elements of each concentric "ring". They were successive powers of odd numbers, e.g. `1=1*1, 9=3*3, 25=5*5`, with the number representing the side length of each ring. Also, every element in a given ring is less than or equal to that corner element (due to the counter-clockwise way the ring is constructed).

With this knowledge, my idea was to figure out on which ring the puzzle input belonged to, figure out its position on the ring, then calculate the Manhattan distance to the center.

In [7]:
input = int(Input(3).read().rstrip('\n'))

dim = math.ceil(math.sqrt(input))
dim

559

So the input sits on the ring with bottom right corner element `559*559==312481`, and where each side has 559 numbers.  `312481 - 312051 == 430` which is less than the side length, so our element sits on the bottom row of the square.

Since the bottom right corner element is the square of an odd number, the number of steps to the center is:

In [8]:
vert_steps = dim // 2
vert_steps

279

Next, to figure out the horizontal distance to the center, we need to take the absolute difference between our input number and the center element in our row. The number in our ring, that is directly below the center `1` element would be:

In [9]:
row_center = dim**2 - dim // 2
row_center

312202

So then the number of horizontal steps would be:

In [10]:
horiz_steps = abs(input - row_center)
horiz_steps

151

So our final answer is:

In [11]:
vert_steps + horiz_steps

430

## Part 2
> If we have a function that can tell us the (x,y) of each item, given the square's index, we could easily compute the values by filling them in sequentially, as we could easily get the neighbours from a 2D array.

Do the values fit a recurrence relation?
Could we figure out a way to map a straight 1d array and coil it?

The size of each concentric "ring" is:
1, 8=2*4, 16=4*4, 24=6*4


1 R 
2 U 3 L 4 L 5 D 6 D 7 R 8 R 9 R
10 U 11 U 12 U 13 L 14 L 15 L 16 L 17 D 18 D 19 D 20 D 21 R 22 R 23 R 24 R 25 R

Make a really large array, e.g. 559x559. Start in middle. Then keep track of size of spiral we're currently filling, and then fill it.



In [12]:
def initialize_2d(rows, cols, init_value = 0):
    return [ [init_value] * cols for r in range(rows) ]

def neighbour_sum(arr, x, y):
    "Return the sum of the adjacent neighbours of arr[x][y], including diagonals"
    rows = len(arr)
    cols = len(arr[0])
    
    if x == (rows // 2) and y == (cols // 2):
        return 1
    
    sum = 0
    if (x-1 >= 0):
        sum += arr[x-1][y]
        if (y-1 >= 0):
            sum += arr[x-1][y-1]
        if (y+1 <= rows-1):
            sum += arr[x-1][y+1]
    if (x+1 <= cols-1):    
        sum += arr[x+1][y]
        if (y-1 >- 0):
            sum += arr[x+1][y-1]
        if (y+1 <= rows-1):
            sum += arr[x+1][y+1]

    if (y-1 >- 0):
        sum += arr[x][y-1]
    if (y+1 <= rows-1):
        sum += arr[x][y+1]

    return sum

In [13]:
def spiral_index_generator(dim):
    generated = 0
    
    x = dim // 2
    y = dim // 2
    yield x, y

    for spiral_len in range(3, dim+1, 2):
        # Each time we start new ring, we are one up
        # from bottom right corner
        dir_steps = 1
        x += 1
        for curdir in ['U', 'L', 'D', 'R']:
            while dir_steps < spiral_len-1:
                yield x, y   

                if curdir == 'U':
                    y -= 1
                elif curdir == 'L':
                    x -= 1
                elif curdir == 'D':
                    y += 1
                elif curdir == 'R':
                    x += 1
                dir_steps += 1
                
            dir_steps = 0
        yield x, y

def stress_test(dim, sentinel):
    arr = initialize_2d(dim, dim)
    for x, y, in spiral_index_generator(dim):
        arr[x][y] = neighbour_sum(arr, x, y)
        if arr[x][y] > sentinel:
            return arr[x][y]

stress_test(15, 312051)

312453

# Day 4
## Part 1
> A passphrase consists of a series of words (lowercase letters) separated by spaces. To ensure security, a valid passphrase must contain no duplicate words. How many passphrases are valid?

The approach I thought of immediately is to maintain a map from the word to a count of the number of times the word appears on each line. But it's simpler to just put all the words in a set, which automatically removes duplicates.

In [14]:
def line_to_string_array(line):
    "Given a string with whitespace-separated words, parse it as an array of strings"
    return [n for n in line.split()]

def lines_to_string_arrays(lines):
    "Given a multi-line string, each with whitespace-separated words, return an array of arrays of words"
    return [line_to_string_array(L) for L in lines]

def duplicate_words(arr):
    "Given an array of words, return true if the array contains duplicate words, false otherwise"
    return len(arr) != len(set(arr))

assert(duplicate_words(line_to_string_array("aa bb cc dd ee"))) == False
assert(duplicate_words(line_to_string_array("aa bb cc dd aa"))) == True
assert(duplicate_words(line_to_string_array("aa bb cc dd aaa"))) == False

valid_lines = [duplicate_words(arr) for arr in lines_to_string_arrays(Input(4).readlines())]

sum([(1 if not valid else 0) for valid in valid_lines])

466

# Day 4
## Part 2
> …a valid passphrase must contain no two words that are anagrams of each other - that is, a passphrase is invalid if any word's letters can be rearranged to form any other word in the passphrase.

One key insight here is that to detect an anagram, you can employ a similar technique to look for duplicate words, but sorting the letters in each word first.

In [15]:
def has_anagram(arr):
    "Given an array of strings, return true if any of the words is an anagram of any other word; false, otherwise."
    dict = { ''.join(sorted([c for c in word])): 1 for word in arr }
    return len(dict.keys()) != len(arr)

assert(has_anagram(line_to_string_array("abcde fghij"))) == False
assert(has_anagram(line_to_string_array("abcde xyz ecdab"))) == True
assert(has_anagram(line_to_string_array("a ab abc abd abf abj"))) == False
assert(has_anagram(line_to_string_array("iiii oiii ooii oooi oooo"))) == False
assert(has_anagram(line_to_string_array("oiii ioii iioi iiio"))) == True

check = [has_anagram(line) for line in lines_to_string_arrays(Input(4).readlines())]

sum(0 if has_anagram else 1 for has_anagram in check)

251

# Day 5
## Part 1
> The message includes a list of the offsets for each jump. Jumps are relative: -1 moves to the previous instruction, and 2 skips the next one. Start at the first instruction in the list. The goal is to follow the jumps until one leads outside the list.

> In addition, these instructions are a little strange; after each jump, the offset of that instruction increases by 1. So, if you come across an offset of 3, you would move three instructions forward, but change it to a 4 for the next time it is encountered.

This seems fairly straightforward: read the data into a mutable array, and just update as we traverse it.

In [16]:
def lines_to_int_array(input):
    return [int(e.strip()) for e in input.split()]

def jump_maze_1(arr):
    "Given an array of integers representing jumps, follow the Part 1 rules and return the number of steps before we reach an index outside of the array dimension."
    steps = 0
    pos = 0
    while True:
        try:
            jump = arr[pos]
            arr[pos] += 1
            pos += jump
            steps += 1
        except IndexError:
            return steps

assert(jump_maze_1(lines_to_int_array("0\n 3\n 0 \n1\n -3"))) == 5

In [17]:
jump_maze_1(lines_to_int_array(Input(5).read()))

342669

## Part 2
> Now, the jumps are even stranger: after each jump, if the offset was three or more, instead decrease it by 1. Otherwise, increase it by 1 as before.

In [18]:
def jump_maze_2(arr):
    "Given an array of integers representing jumps, follow the Part 2 rules and return the number of steps before we reach an index outside of the array dimension."
    steps = 0
    pos = 0
    while True:
        try:
            jump = arr[pos]
            offset = -1 if jump >= 3 else 1
            arr[pos] += offset
            pos += jump
            steps += 1
        except IndexError:
            return steps
assert(jump_maze_2(lines_to_int_array("0\n 3\n 0 \n1\n -3"))) == 10

In [19]:
jump_maze_2(lines_to_int_array(Input(5).read()))

25136209

# Day 6
## Part 1
> The reallocation routine operates in cycles. In each cycle, it finds the memory bank with the most blocks (ties won by the lowest-numbered memory bank) and redistributes those blocks among the banks. To do this, it removes all of the blocks from the selected bank, then moves to the next (by index) memory bank and inserts one of the blocks. It continues doing this until it runs out of blocks; if it reaches the last memory bank, it wraps around to the first one.

> The debugger would like to know how many redistributions can be done before a blocks-in-banks configuration is produced that has been seen before.

The obvious solution is to just follow the description and keep track of the index of the current position in the memory bank, a counter for how many redistribution cycles have been performed, and a map of the states seen. To do a redistribution, find `max(array)` and use `array.index(max(array))` to know where to start redistribution.

Improving on the obvious solution, instead of adding one to each bank during redistribution, try to update each bank with all the blocks that it needs, by dividing the selected block value by the size of the array. There's a bit of adjustment to account for the remainder if the division is not exact.

## Part 2
> starting from a state that has already been seen, how many block redistribution cycles must be performed before that same state is seen again?

This can be done by extending the redistribution implementation, and storing the cycle number where a state was first seen. When we see it again, we subtract the current cycle number from the cycle when it was first seen, to determine the size of the loop.

In [20]:
def redistribute(arr):
    alen = len(arr)
    pos = 0
    cycles = 0
    states = {}
    while True:
        most = max(arr)
        pos = arr.index(most) # finds lowest index
        arr[pos] = 0 # Remove for redistribution
        pos += 1
        while most > 0:
            arr[pos % alen] += 1
            most -= 1
            pos += 1
        cycles += 1
        #print(tuple(arr))
        if (tuple(arr) in states):
            first_repeat_state = tuple(arr)
            break
        states[tuple(arr)] = cycles # Records on which cycle this state was first seen
    return cycles, (first_repeat_state, states[first_repeat_state])

cycles, (state, cycle_first_seen) = redistribute([0,2,7,0])
assert(cycles) == 5
assert(cycles - cycle_first_seen) == 4

In [21]:
cycles, (first_repeat_state, cycle_first_seen) = redistribute(line_to_int_array(Input(6).read()))
cycles

3156

In [22]:
cycles - cycle_first_seen

1610

# Day 7
## Part 1

> One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own.
> You ask each program to yell out their name, their weight, and (if they're holding a disc) the names of the programs immediately above them balancing on that disc. 
> What is the name of the bottom program?

In [102]:
import re
def find_bottom_program(lines):
    parentmap = {}
    for line in lines:
        m = re.match(r"(\w+) \((\d+)\)( -> (.*))?", line)
        # Ignore leaf nodes, they don't help in finding lowest
        if m != None and m.group(4) != None:
            parentname = m.group(1)
            children = [e.strip() for e in m.group(4).split(',')]
            for childname in children:
                parentmap[childname] = parentname
    
    # Pick arbitrary node, traverse up to parent
    visit = list(parentmap.keys())[0]
    while visit in parentmap:
        visit = parentmap[visit]
    return visit

test_data = """
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
"""
assert(find_bottom_program(test_data.split('\n'))) == 'tknk'


In [103]:
find_lowest(Input(7).readlines())

'cyrupz'

## Part 2
> For any program holding a disc, each program standing on that disc forms a sub-tower. Each of those sub-towers are supposed to be the same weight, or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower.
> Given that exactly one program is the wrong weight, what would its weight need to be to balance the entire tower?

Store all the weights of each disc in one map. Track each disc's children in another map. Define a recursive function to compute weight of program on a disc: if leaf node (no children), weight is own weight. If children, then own weight plus sum of childrens' weights. 

In [13]:
def initialize_tower_structures(lines):
    weight_map = {}
    child_map = {}
    for line in lines:
        m = re.match(r"(\w+) \((\d+)\)( -> (.*))?", line)
        name = m.group(1)
        weight = int(m.group(2))
        weight_map[name] = weight
        
        if m.group(4) != None:
            child_map[name] = [e.strip() for e in m.group(4).split(',')]
    return weight_map, child_map

In [14]:
def program_weight(name, weight_map, child_map):
    self = weight_map[name]
    children = (sum([program_weight(child, weight_map, child_map) for child in child_map[name]])) if name in child_map else 0
    return self + children

In [15]:
weight_map, child_map = initialize_tower_structures( Input(7).readlines() )

Starting from root (found in previous part), we check weights of children and recurse on whichever has a different weight than the others. Once we have found the disc that is balanced, we look at its peers to determine what weight needs to be adjusted.

In [38]:
import collections

def find_black_sheep(L, common):
    "Given an array of items that are all assumed to be the given value 'common', return the index of the odd one out. If all elements are the same, raise StopIteration"
    return next(i for i, v in enumerate(L) if v != common)

def find_weight_adjustment(root_name, weight_map, child_map):
    print(root_name)
    child_weights = [program_weight(child, weight_map, child_map) for child in child_map[root_name]]
    print("\t{}".format(child_weights))
    try:
        common = collections.Counter(child_weights).most_common()[0][0]
        index = find_black_sheep(child_weights, common)
        next_root = child_map[root_name][index]
        adjust = find_weight_adjustment(next_root, weight_map, child_map)
        if adjust == None:
            next_root_weight = weight_map[next_root]
            black_sheep = child_weights[index]
            print("weight[{}]={} + ({} - {})".format(next_root, next_root_weight, common, black_sheep))
            return next_root_weight + (common - black_sheep)
        else:
            return adjust
    except StopIteration:
        return None

In [39]:
find_weight_adjustment('cyrupz', weight_map, child_map)

cyrupz
	[119424, 119424, 119424, 119424, 119424, 119424, 119432]
qjvtm
	[12146, 12154, 12146]
boropxd
	[1123, 1123, 1123, 1123, 1131, 1123, 1123]
cwwwj
	[310, 310, 310]
weight[cwwwj]=201 + (1123 - 1131)


193

# Day 8
## Part 1
> Each instruction consists of several parts: the register to modify, whether to increase or decrease that register's value, the amount by which to increase or decrease it, and a condition. If the condition fails, skip the instruction without modifying the register. The registers all start at 0. The instructions look like this:
```
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10
```

Keep a map of all registers. Parse each instruction line, check the condition, and execute.

In [106]:
from collections import defaultdict
def parse_instruction(line):
    m = re.match(r"(\w+) (inc|dec) (-?\d+) if (\w+) (.*) (-?\d+)", line)
    return m.groups()

def test_reg_condition(regs, test_reg, oper, str_value):
    "Return the result of evaluating a register predicate."
    val = int(str_value)
    if oper == '>':
        return regs[test_reg] > val
    elif oper == '<':
        return regs[test_reg] < val
    elif oper == '>=':
        return regs[test_reg] >= val
    elif oper == '<=':
        return regs[test_reg] <= val
    elif oper == '==':
        return regs[test_reg] == val
    elif oper == '!=':
        return regs[test_reg] != val
    else:
        raise Exception
    
def compute_result(lines, regs):
    "Execute the program given in 'lines', updating register values in 'regs', and returning the largest value ever stored in any register."
    highwater = 0
    for line in lines:
        reg, inc_oper, val, test_reg, test_oper, test_value = parse_instruction(line)
        if test_reg_condition(regs, test_reg, test_oper, test_value):
            value = 0
            if inc_oper == 'inc':
                value = int(val)
            elif inc_oper == 'dec':
                value = -int(val)
            regs[reg] += value
            if regs[reg] > highwater:
                highwater = regs[reg]
    return highwater

regs = defaultdict(lambda: 0)

test = """b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10"""
highwater = compute_result(test.split('\n'), regs)
assert(max(regs.values())) == 1
assert(highwater) == 10

regs = defaultdict(lambda: 0)
compute_result(Input(8).readlines(), regs)
max(regs.values())

4066

## Part 2
> the CPU also needs to know the highest value held in any register during this process

In [107]:
regs = defaultdict(lambda: 0)
compute_result(Input(8).readlines(), regs)

4829

# Day 9
## Part 1
> The characters represent groups - sequences that begin with { and end with }. Within a group, there are zero or more other things, separated by commas: either another group or garbage. Since groups can contain other groups, a } only closes the most-recently-opened unclosed group - that is, they are nestable. Your puzzle input represents a single, large group which itself contains many smaller ones.

> Sometimes, instead of a group, you will find garbage. Garbage begins with < and ends with >. Between those angle brackets, almost any character can appear, including { and }. Within garbage, < has no special meaning… Inside garbage, any character that comes after ! should be ignored, including <, >, and even another !.

Process groups recursively using state machine, character by character. 
Parent group decides on score to pass into child group
{ means start of group
Group processing keeps track of own score
Each time new group started, increment score to pass in
Once in garbage, only thing to look for is ending >

In [43]:
from io import StringIO
def parse_garbage(file):
    "Given a stream pointing at the character right after the start of the garbage start character, continue reading until the garbage block is completely read. The stream will be left at one character after the end garbage character. Return the number of garbage characters, not including the start/end chars nor cancelled chars nor the cancelling chars."
    count = 0
    char = read_char(file)
    while char == '!' and char != '':
        read_char(file)
        char = read_char(file)
    while char != '>' and char != '':
        count += 1
        char = read_char(file)
        while char == '!' and char != '':
            read_char(file)
            char = read_char(file)
    return count
    
def parse_group(own_score, file):
    "Given a file object, read from the current position until we see an ending group character. Return the group score."
    garbage_count = 0
    score = own_score
    char = read_char(file)
    while char != '}' and char != '':
        if char == '<':
            garbage_count += parse_garbage(file)
        elif char == '{':
            child_group_score, child_garbage_count = parse_group(own_score+1, file)
            score += child_group_score
            garbage_count += child_garbage_count
         
        char = read_char(file)

    return score, garbage_count

def read_char(file):
    char = file.read(1)
    return char

assert(parse_group(0, StringIO('{}'))[0]) == 1
assert(parse_group(0, StringIO('{{{}}}'))[0]) == 6
assert(parse_group(0, StringIO('{{},{}}'))[0]) == 5
assert(parse_group(0, StringIO('{{{},{},{{}}}}'))[0]) == 16
assert(parse_group(0, StringIO('{<a>,<a>,<a>,<a>}'))[0]) == 1
assert(parse_group(0, StringIO('{{<ab>},{<ab>},{<ab>},{<ab>}}'))[0]) == 9
assert(parse_group(0, StringIO('{{<!!>},{<!!>},{<!!>},{<!!>}}'))[0]) == 9
assert(parse_group(0, StringIO('{{<a!>},{<a!>},{<a!>},{<ab>}}'))[0]) == 3


In [44]:
parse_group(0, Input(9))

(12803, 6425)